# LangChain Location Tool

In [124]:
# Models and Env
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

# Agent and Tools
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.tools import BaseTool, format_tool_to_openai_function
from pydantic import BaseModel, Field
from typing import Optional, Type
import requests
import json
import os

# Message Schemas
from langchain.schema import HumanMessage, FunctionMessage

In [125]:
# Import API Keys

load_dotenv()

openai_key = os.environ.get("OPENAI_API_KEY")
gplaces_key = os.environ.get("GOOGLE_MAPS_API_KEY")

# Setting the Model

model_name = "gpt-3.5-turbo"

## Functions

In [126]:
# Function for Geocoding API

def get_coordinates(place):

    API_KEY = gplaces_key 

    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={place}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        latitude = location["lat"]
        longitude = location["lng"]
        return f"{latitude}, {longitude}"
        
    else:
        print("Geocoding request failed")
        return None, None

In [127]:
# Function for Google Places API

def get_location_info(location, type, keyword):
    
    API_KEY = gplaces_key
    BASE_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    results = []
    
    coordinates = get_coordinates(location)

    params = {
        'location': coordinates,
        'type': type,
        'keyword': keyword,
        'radius': 1000,
        'key': API_KEY
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if 'results' in data:
        for place in data['results']:
            name = place['name']
            rating = place.get('rating', 'N/A')
            vicinity = place.get('vicinity', 'N/A')
            results.append({'name': name, 'rating': rating, 'vicinity': vicinity})
    else:
        print("No results found.")

    return results

## BaseModel and BaseTool

In [128]:
# Represents input data structure

class GetLocationInfoCheckInput(BaseModel):
    
    location: str = Field(..., description = "The name of the area for which we need to find the type of place in.")
    type: str = Field(..., description = "Information about what type of place it is. For example, is it a restaurant, a gym, a school.")
    keyword: str = Field(..., description = "A specific word or term to refine the search results. This can be used to filter places based on a specific theme or characteristic.")

In [129]:
# The tool that will be used

class GetLocationInfoTool(BaseTool):
    
    name = "get_location_info"
    description = "Used to get information about a given location"
    
    # Executes the tool
    def _run(self, location: str, type: str, keyword: str):
        location_response = get_location_info(location, type, keyword)
        return location_response
    
    def _arun(self, location: str, type:str, keyword: str):
        raise NotImplementedError("This tool does not support async")
   
    args_schema: Optional[Type[BaseModel]] = GetLocationInfoCheckInput

## Setting Up the Tool

In [130]:
# Converts the tool to an OpenAI function

tools = [GetLocationInfoTool()]
functions = [format_tool_to_openai_function(tool_name) for tool_name in tools]

In [131]:
tools[0]

GetLocationInfoTool(name='get_location_info', description='Used to get information about a given location', args_schema=<class '__main__.GetLocationInfoCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False)

In [132]:
functions

[{'name': 'get_location_info',
  'description': 'Used to get information about a given location',
  'parameters': {'title': 'GetLocationInfoCheckInput',
   'type': 'object',
   'properties': {'location': {'title': 'Location',
     'description': 'The name of the area for which we need to find the type of place in.',
     'type': 'string'},
    'type': {'title': 'Type',
     'description': 'Information about what type of place it is. For example, is it a restaurant, a gym, a school.',
     'type': 'string'},
    'keyword': {'title': 'Keyword',
     'description': 'A specific word or term to refine the search results. This can be used to filter places based on a specific theme or characteristic.',
     'type': 'string'}},
   'required': ['location', 'type', 'keyword']}}]

## Using the Tool

In [133]:
# User Query

query = input("What place are you looking for?")

In [134]:
query

'Best burger joints around Makati, Philippines'

In [135]:
# Chat Model

model = ChatOpenAI(
            model = model_name, 
            temperature = 0,
        )

# First Response: Passed the Query and Function to OpenAI
response_ai_message = model.predict_messages(
                        [HumanMessage(content = query)], 
                        functions = functions
                    )

response_ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_location_info', 'arguments': '{\n  "location": "Makati, Philippines",\n  "type": "restaurant",\n  "keyword": "burger"\n}'}}, example=False)

In [136]:
response_ai_message.additional_kwargs['function_call']

{'name': 'get_location_info',
 'arguments': '{\n  "location": "Makati, Philippines",\n  "type": "restaurant",\n  "keyword": "burger"\n}'}

In [137]:
# Assigned the arguments from response_ai_message to _args

_args = json.loads(response_ai_message.additional_kwargs['function_call'].get('arguments'))

_args

{'location': 'Makati, Philippines', 'type': 'restaurant', 'keyword': 'burger'}

In [138]:
tools[0]

GetLocationInfoTool(name='get_location_info', description='Used to get information about a given location', args_schema=<class '__main__.GetLocationInfoCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False)

In [139]:
# Using the tool: _args to the GetLocationInfoTool

tool_result = tools[0](_args)

tool_result

[{'name': 'Brgr: The Burger Project',
  'rating': 0,
  'vicinity': '38 Jupiter, Makati'},
 {'name': 'Moss Burger',
  'rating': 4.3,
  'vicinity': 'H22G+J5C, Glorietta Pedestrian Walkway, Makati'},
 {'name': 'Fuel Burgers',
  'rating': 3.1,
  'vicinity': 'SM Makati, Palm Dr, Makati'},
 {'name': '8 Cuts Burger Blends',
  'rating': 4.5,
  'vicinity': 'H.V. dela Costa St. cor L.P. Leviste St., Salcedo village, Makati, 1227 Metro Manila, L.P. Leviste Street, Makati'},
 {'name': 'MOS Burger',
  'rating': 4.1,
  'vicinity': 'H22G+74Q, Palm Dr, Makati'},
 {'name': 'Mr Beast Burger', 'rating': 5, 'vicinity': 'H23F+RQ2, Makati'},
 {'name': 'Burger Machine',
  'rating': 3.3,
  'vicinity': 'H237+W6X, V.A. Rufino St, Makati'},
 {'name': 'Shake Shack',
  'rating': 4.3,
  'vicinity': 'Level 1, Greenbelt Dr, Makati'},
 {'name': 'Burger Machine',
  'rating': 3.3,
  'vicinity': 'Euro Hotel, Arnaiz Ave Cor. Chino Roces Avenue, Makati'},
 {'name': 'Buns & Burgers',
  'rating': 5,
  'vicinity': '2nd floor,

In [140]:
FunctionMessage(name = "get_location_info", content = str(tool_result))

FunctionMessage(content='[{\'name\': \'Brgr: The Burger Project\', \'rating\': 0, \'vicinity\': \'38 Jupiter, Makati\'}, {\'name\': \'Moss Burger\', \'rating\': 4.3, \'vicinity\': \'H22G+J5C, Glorietta Pedestrian Walkway, Makati\'}, {\'name\': \'Fuel Burgers\', \'rating\': 3.1, \'vicinity\': \'SM Makati, Palm Dr, Makati\'}, {\'name\': \'8 Cuts Burger Blends\', \'rating\': 4.5, \'vicinity\': \'H.V. dela Costa St. cor L.P. Leviste St., Salcedo village, Makati, 1227 Metro Manila, L.P. Leviste Street, Makati\'}, {\'name\': \'MOS Burger\', \'rating\': 4.1, \'vicinity\': \'H22G+74Q, Palm Dr, Makati\'}, {\'name\': \'Mr Beast Burger\', \'rating\': 5, \'vicinity\': \'H23F+RQ2, Makati\'}, {\'name\': \'Burger Machine\', \'rating\': 3.3, \'vicinity\': \'H237+W6X, V.A. Rufino St, Makati\'}, {\'name\': \'Shake Shack\', \'rating\': 4.3, \'vicinity\': \'Level 1, Greenbelt Dr, Makati\'}, {\'name\': \'Burger Machine\', \'rating\': 3.3, \'vicinity\': \'Euro Hotel, Arnaiz Ave Cor. Chino Roces Avenue, Maka

In [141]:
# Second Response: Included the FunctionMessage to the message to OpenAI

response_final = model.predict_messages(
                    [
                        HumanMessage(content = query),
                        response_ai_message,
                        FunctionMessage(name = 'get_location_info',content = str(tool_result)),
                    ], 
                    functions=functions
                )
response_final

AIMessage(content='Here are some of the best burger joints around Makati, Philippines:\n\n1. Brgr: The Burger Project - Located at 38 Jupiter, Makati. Rating: 0/5.\n\n2. Moss Burger - Located at H22G+J5C, Glorietta Pedestrian Walkway, Makati. Rating: 4.3/5.\n\n3. Fuel Burgers - Located at SM Makati, Palm Dr, Makati. Rating: 3.1/5.\n\n4. 8 Cuts Burger Blends - Located at H.V. dela Costa St. cor L.P. Leviste St., Salcedo village, Makati. Rating: 4.5/5.\n\n5. MOS Burger - Located at H22G+74Q, Palm Dr, Makati. Rating: 4.1/5.\n\n6. Mr Beast Burger - Located at H23F+RQ2, Makati. Rating: 5/5.\n\n7. Burger Machine - Located at H237+W6X, V.A. Rufino St, Makati. Rating: 3.3/5.\n\n8. Shake Shack - Located at Level 1, Greenbelt Dr, Makati. Rating: 4.3/5.\n\n9. Buns & Burgers - Located at 2nd floor, 8445 Kalayaan Ave, Manila. Rating: 5/5.\n\n10. ArmyNavy Burger + Burrito - Located at 2nd Floor, New Glorietta, Ayala Center San Lorenzo Village, Makati. Rating: 3.2/5.\n\nPlease note that these ratings

In [142]:
output = response_final

In [143]:
print(output)

content='Here are some of the best burger joints around Makati, Philippines:\n\n1. Brgr: The Burger Project - Located at 38 Jupiter, Makati. Rating: 0/5.\n\n2. Moss Burger - Located at H22G+J5C, Glorietta Pedestrian Walkway, Makati. Rating: 4.3/5.\n\n3. Fuel Burgers - Located at SM Makati, Palm Dr, Makati. Rating: 3.1/5.\n\n4. 8 Cuts Burger Blends - Located at H.V. dela Costa St. cor L.P. Leviste St., Salcedo village, Makati. Rating: 4.5/5.\n\n5. MOS Burger - Located at H22G+74Q, Palm Dr, Makati. Rating: 4.1/5.\n\n6. Mr Beast Burger - Located at H23F+RQ2, Makati. Rating: 5/5.\n\n7. Burger Machine - Located at H237+W6X, V.A. Rufino St, Makati. Rating: 3.3/5.\n\n8. Shake Shack - Located at Level 1, Greenbelt Dr, Makati. Rating: 4.3/5.\n\n9. Buns & Burgers - Located at 2nd floor, 8445 Kalayaan Ave, Manila. Rating: 5/5.\n\n10. ArmyNavy Burger + Burrito - Located at 2nd Floor, New Glorietta, Ayala Center San Lorenzo Village, Makati. Rating: 3.2/5.\n\nPlease note that these ratings are based

## Using Agents

In [144]:
model_using_agent = ChatOpenAI(temperature = 0, model = model_name)

In [145]:
agent_for_function = initialize_agent(
                        tools,
                        model_using_agent,
                        agent = AgentType.OPENAI_FUNCTIONS,
                        verbose = True
                    )

In [146]:
final_output = agent_for_function.run(query)



> Entering new AgentExecutor chain...

Invoking: `get_location_info` with `{'location': 'Makati, Philippines', 'type': 'restaurant', 'keyword': 'burger'}`


[{'name': 'Brgr: The Burger Project', 'rating': 0, 'vicinity': '38 Jupiter, Makati'}, {'name': 'Moss Burger', 'rating': 4.3, 'vicinity': 'H22G+J5C, Glorietta Pedestrian Walkway, Makati'}, {'name': 'Fuel Burgers', 'rating': 3.1, 'vicinity': 'SM Makati, Palm Dr, Makati'}, {'name': '8 Cuts Burger Blends', 'rating': 4.5, 'vicinity': 'H.V. dela Costa St. cor L.P. Leviste St., Salcedo village, Makati, 1227 Metro Manila, L.P. Leviste Street, Makati'}, {'name': 'MOS Burger', 'rating': 4.1, 'vicinity': 'H22G+74Q, Palm Dr, Makati'}, {'name': 'Mr Beast Burger', 'rating': 5, 'vicinity': 'H23F+RQ2, Makati'}, {'name': 'Burger Machine', 'rating': 3.3, 'vicinity': 'H237+W6X, V.A. Rufino St, Makati'}, {'name': 'Shake Shack', 'rating': 4.3, 'vicinity': 'Level 1, Greenbelt Dr, Makati'}, {'name': 'Burger Machine', 'rating': 3.3, 'vicinity': 'Euro Ho

In [147]:
print(final_output)

Here are some of the best burger joints around Makati, Philippines:

1. Brgr: The Burger Project - Located at 38 Jupiter, Makati. (Rating: 0)

2. Moss Burger - Located at H22G+J5C, Glorietta Pedestrian Walkway, Makati. (Rating: 4.3)

3. Fuel Burgers - Located at SM Makati, Palm Dr, Makati. (Rating: 3.1)

4. 8 Cuts Burger Blends - Located at H.V. dela Costa St. cor L.P. Leviste St., Salcedo village, Makati. (Rating: 4.5)

5. MOS Burger - Located at H22G+74Q, Palm Dr, Makati. (Rating: 4.1)

6. Mr Beast Burger - Located at H23F+RQ2, Makati. (Rating: 5)

7. Burger Machine - Located at H237+W6X, V.A. Rufino St, Makati. (Rating: 3.3)

8. Shake Shack - Located at Level 1, Greenbelt Dr, Makati. (Rating: 4.3)

9. Buns & Burgers - Located at 2nd floor, 8445 Kalayaan Ave, Manila. (Rating: 5)

10. ArmyNavy Burger + Burrito - Located at 2nd Floor, New Glorietta, Ayala Center San Lorenzo Village, Makati. (Rating: 3.2)

Please note that the ratings are based on user reviews and may vary. Enjoy your b